__Standalone notebook for cleaning Box Office Mojo data__


In [23]:
# Import the required libraries
import pandas as pd
import numpy as np

# set up pandas to display floats in a more human friendly way
pd.options.display.float_format = '{:,.2f}'.format

In [24]:
bom_movie_gross_df = pd.read_csv('./zippedData/bom.movie_gross.csv.gz')
bom_movie_gross_df.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,"415,000,000.00",652000000,2010
1,Alice in Wonderland (2010),BV,"334,200,000.00",691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,"296,000,000.00",664300000,2010
3,Inception,WB,"292,600,000.00",535700000,2010
4,Shrek Forever After,P/DW,"238,700,000.00",513900000,2010


Check for duplicates in the data

In [266]:
bom_dups = bom_movie_gross_df[bom_movie_gross_df.duplicated()]
print(len(bom_dups))

0


In [12]:
print(bom_movie_gross_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB
None


3359 of 3387 records had domestic_gross, 

2037 of 3387 had foreign_gross, 

so only 2037 could be used to create worldwide_gross. 

domestric_gross is already in numeric form. year is an int not a date, might be fine to group by year.

DATA CLEANING - 

title no NAs
year no NaNs

studio 5 had NaNs...replace them with "UNKNOWN" for unknown

foreign_gross is str neeeds to be converted to float


domestic_gross 3359 out of 3387...will select a subset to use

foreign_gross  2037 out of 3387.. will have to subset to use

making a worldwide_gross where there are values for both domestic_gross and foreign_gross in a row

In [13]:
bom_movie_gross_df['studio'].fillna('UNKNOWN', inplace=True)

In [14]:
bom_movie_gross_df['foreign_gross'] = bom_movie_gross_df['foreign_gross'].str.replace(',', '').astype(float)

In [15]:
print(bom_movie_gross_df.describe())

       domestic_gross  foreign_gross     year
count        3,359.00       2,037.00 3,387.00
mean    28,745,845.07  74,872,810.15 2,013.96
std     66,982,498.24 137,410,600.84     2.48
min            100.00         600.00 2,010.00
25%        120,000.00   3,700,000.00 2,012.00
50%      1,400,000.00  18,700,000.00 2,014.00
75%     27,900,000.00  74,900,000.00 2,016.00
max    936,700,000.00 960,500,000.00 2,018.00


Creating a new dataframe that only includes movies with values for both domestic and foreign gross so we can calculate worldwide gross.

There are 2009 movies in this dataset

In [16]:
bom_ww_gross_df = bom_movie_gross_df.loc[bom_movie_gross_df['domestic_gross'].notnull() & bom_movie_gross_df['foreign_gross'].notnull()]
bom_ww_gross_df['worldwide_gross'] = bom_ww_gross_df['domestic_gross'] + bom_ww_gross_df['foreign_gross']

/home/user/anaconda3/envs/flatiron/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [17]:
print(bom_ww_gross_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2009 entries, 0 to 3353
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title            2009 non-null   object 
 1   studio           2009 non-null   object 
 2   domestic_gross   2009 non-null   float64
 3   foreign_gross    2009 non-null   float64
 4   year             2009 non-null   int64  
 5   worldwide_gross  2009 non-null   float64
dtypes: float64(3), int64(1), object(2)
memory usage: 109.9+ KB
None


In [18]:
print(bom_ww_gross_df.describe())

       domestic_gross  foreign_gross     year  worldwide_gross
count        2,009.00       2,009.00 2,009.00         2,009.00
mean    46,973,114.38  75,718,219.20 2,013.50   122,691,333.58
std     81,599,661.11 138,129,649.08     2.60   207,487,031.92
min            400.00         600.00 2,010.00         4,900.00
25%        665,000.00   3,900,000.00 2,011.00     8,141,000.00
50%     16,500,000.00  19,300,000.00 2,013.00    42,300,000.00
75%     56,000,000.00  75,900,000.00 2,016.00   133,700,000.00
max    936,700,000.00 960,500,000.00 2,018.00 1,518,900,000.00


Our subset of movies with both domestic and foreign gross includes movies released during 2010-2018.

In [19]:
bom_ww_gross_df['year'].unique()

array([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018])

__SAVE CLEANED BOM data__

In [21]:
bom_ww_gross_df.to_csv('../cleanedData/bom.worldwide_gross.csv', index=False)